In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np
import torch
from torch import nn
from scipy.stats import lognorm
from matplotlib import pyplot as plt
from scipy.ndimage.filters import uniform_filter1d
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from datetime import date, datetime
from torch.utils.data import DataLoader

In [3]:
#integration parameter
INTERVAL_LENGTH = 10
START_UNIF = 0.01
SCALING_CONSTANT = 10000

In [5]:
options = pd.read_csv("../input/jpx-tokyo-stock-exchange-prediction/train_files/options.csv")
print(options.columns)


/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (7,8,9,10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Index(['DateCode', 'Date', 'OptionsCode', 'WholeDayOpen', 'WholeDayHigh',
       'WholeDayLow', 'WholeDayClose', 'NightSessionOpen', 'NightSessionHigh',
       'NightSessionLow', 'NightSessionClose', 'DaySessionOpen',
       'DaySessionHigh', 'DaySessionLow', 'DaySessionClose', 'TradingVolume',
       'OpenInterest', 'TradingValue', 'ContractMonth', 'StrikePrice',
       'WholeDayVolume', 'Putcall', 'LastTradingDay', 'SpecialQuotationDay',
       'SettlementPrice', 'TheoreticalPrice', 'BaseVolatility',
       'ImpliedVolatility', 'InterestRate', 'DividendRate', 'Dividend'],
      dtype='object')


In [6]:
datasets = []
for trading_date in tqdm(options['Date'].unique()):
    dataset = options.loc[options['Date'] == trading_date]
    for option_expiry in dataset['LastTradingDay'].unique():
        dataset = dataset.loc[(options['LastTradingDay'] == option_expiry)& (options['WholeDayVolume']> 0)][['StrikePrice', 'Putcall', 'SettlementPrice']]#[['WholeDayClose', 'SettlementPrice']]
        if dataset.empty:
            continue

        dataset[['StrikePrice', 'SettlementPrice']] = dataset[['StrikePrice', 'SettlementPrice']].transform(lambda x : x/SCALING_CONSTANT)
        dataset['Putcall'] = dataset['Putcall'].transform(lambda x : False if x==1 else True)
        trading_date = datetime.strptime(trading_date, '%Y-%m-%d')
        option_expiry = datetime.strptime(str(option_expiry), '%Y%m%d')
        ttm = option_expiry - trading_date
        datasets.append((dataset, ttm.days))

100%|██████████| 1202/1202 [15:11<00:00,  1.32it/s]


In [7]:
import pickle

# Step 2
with open('/kaggle/working/datasets.pkl', 'wb') as datasets_file:
 
  # Step 3
  pickle.dump(datasets, datasets_file)


In [24]:
ttms = [dataset[1] for dataset in datasets]
lengths = [len(dataset[0]) for dataset in datasets]

In [ ]:
"""


import pickle
with open('/kaggle/working/datasets.pkl', 'rb') as datasets_file:
    datasets_copy = pickle.load(datasets_file)


"""

In [ ]:
class TimeEncodingNet(nn.Module):
    def __init__(out_size, self):
        self.out_size = out_size
        self.mlp = nn.Sequential(
            nn.Linear(2, 16),
            nn.ReLU(),
            nn.Linear(16, 32),
            nn.ReLU(),
            nn.Linear(32, out_size)
        )
        
    def forward(self, ttm):
        return self.mlp((ttm, torch.sqrt(ttm) ) )



In [ ]:
class ImprovedNet(nn.Module):
    def __init__(time_encoding_net, self):
        super(BaselineNet, self).__init__()
        self.time_encoding_net = time_encoding_net
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(2, 128),
            nn.LeakyReLU(1e-1),
            
            nn.Linear(128, 512),
            nn.LeakyReLU(1e-1),   

            nn.Linear(512, 128),
            nn.LeakyReLU(1e-1),
            
            nn.Linear(128, self.time_encoding_net.out_size)
        )
        self.common_head = nn.Sequential(
            nn.Linear(2*self.time_encoding_net.out_size, 64),
            nn.LeakyReLU(1e-1),
            nn.Linear(64, 1)
        )
        self.softplus = nn.Softplus()
        
        
    def forward(self, x, ttm):
        x = self.flatten(x)
        y = torch.log(x)
        x = self.linear_relu_stack(torch.cat((x,y), 1))
        x = torch.cat((x, self.time_encoding_net(ttm)), 1)
        x = self.common_head(x)
        x = self.softplus(x)
        return x

In [ ]:
class OptionLoss(nn.Module):
    def __init__(self):
        super(OptionLoss,self).__init__()
        self.huber = nn.HuberLoss()
    
    # outputs the MC_STEPS predictions, labels the TOT_OPTIONS labels
    def forward(self, outputs, labels, strike, is_call, coordinates):
        mc_steps = len(coordinates)
        loss = 0
        mc_integral = 0
        payoff = lambda  j :  torch.max(torch.zeros(len(outputs)), coordinates - strike[j] ) if is_call[j] else torch.max(torch.zeros(len(outputs)), strike[j] - coordinates)
        density = 1/INTERVAL_LENGTH 
        mc_prices = torch.stack([ torch.sum(payoff(j) * outputs / density)/mc_steps for j in range(len(labels))])
        
        return self.huber(mc_prices/labels, torch.ones_like(labels))


In [ ]:
class AbsolutePercentageLoss(nn.Module):
    def __init__(self):
        super(AbsolutePercentageLoss, self).__init__()
    
    # outputs the MC_STEPS predictions, labels the TOT_OPTIONS labels
    def forward(self, outputs, labels, strike, is_call, coordinates):
        mc_steps = len(coordinates)
        loss = 0
        mc_integral = 0
        payoff = lambda  j :  torch.max(torch.zeros(len(outputs)), coordinates - strike[j] ) if is_call[j] else torch.max(torch.zeros(len(outputs)), strike[j] - coordinates)
        densities = 1/INTERVAL_LENGTH 
        for j in range(len(labels)):
          mc_price = payoff(j) * outputs / densities
          #print("true price " + str(labels[j]) )
          #print("predicted price " + str(mc_price.sum() / mc_steps) )
          #print("absolute difference " + str(torch.abs(labels[j] -  mc_price.sum() / mc_steps)))
          print("delta (percentage of true price)" + str(torch.abs(labels[j] -  mc_price.sum() / mc_steps)/labels[j]) )
          loss += torch.abs(labels[j] -  mc_price.sum() / mc_steps)/labels[j]
        return loss  

In [ ]:
time_encoding_net = TimeEncodingNet(64)
improved_model = ImprovedNet(time_encoding_net)
optimizer_whole_network = torch.optim.Adam(improved_model.parameters(), lr=1e-4, weight_decay=1e-20)


In [ ]:
def train_improved_model(input_model, optimizer, loss_fn, C1, C2, target_dataloader, pretrain_datasets, pretrain_iterations=3, pretrain_epochs = 5, equidistant_window=False):
    mc_steps = 256 #fixed for pretraining
    for _ in range(pretrain_iterations):
        for dataset in pretrain_datasets:
            prices_and_strikes = [ (dataset['SettlementPrice'].iloc[i], dataset['StrikePrice'].iloc[i], dataset['Putcall'].iloc[i]) for i in range(len(dataset))]
            train_set_size = len(prices_and_strikes)
            batch_size = train_set_size
            target_dataloader = DataLoader(prices_and_strikes , batch_size=batch_size, shuffle = True)
            for epoch in pretrain_epochs:
                X = torch.zeros(mc_steps, 1).to(device)
                X.uniform_(START_UNIF, INTERVAL_LENGTH)
                pred = input_model(X)
                loss = loss_fn(pred.cpu().squeeze(), target_prices, strike_prices, is_call, X.cpu().squeeze())
                optimizer.zero_grad()
                loss.backward()
                torch.nn.utils.clip_grad_norm_(input_model.parameters(), 1e-1)
                optimizer.step()
    
    for param in input_model.time_encoding_net.parameters(): 
        param.requires_grad = False #freeze layers of the time encoding net
    
    loss_history = []
    for epoch in tqdm(range(int(600*C1))):
        avg_loss = 0
        for batch, (target_prices, strike_prices, is_call) in enumerate(target_dataloader):
            if epoch < 100 * C1:
                mc_steps = int(128*C2)
            elif epoch < 200 * C1:
                mc_steps = int(256*C2)
            elif epoch < 250 * C1:
                mc_steps = int(512*C2)
            elif epoch < 275 * C1:
                mc_steps = int(1024*C2)
            elif epoch < 287 * C1:
                mc_steps = int(2048*C2)
            elif epoch < 293*C1:
                mc_steps = int(4096*C2)
            else:
                mc_steps = int(8192*C2)
            
            
            if equidistant_window:
                coordinates=np.linspace(start=START_UNIF+INTERVAL_LENGTH/(2*mc_steps),stop=START_UNIF+INTERVAL_LENGTH-INTERVAL_LENGTH/(2*mc_steps),num=mc_steps)+np.random.uniform(low=-INTERVAL_LENGTH/(2*mc_steps),high=INTERVAL_LENGTH/(2*mc_steps),size=mc_steps)
                coordinates = coordinates.reshape(  -1, 1)
                X = torch.tensor(coordinates, dtype = torch.float32).to(device)
                

            else:
                X = torch.zeros(mc_steps, 1).to(device)
                X.uniform_(START_UNIF, INTERVAL_LENGTH)



            #print(X)
            #X.log_normal_(PROPOSAL_MU, PROPOSAL_SIGMA )
            #print(X)
            pred = input_model(X)
            loss = loss_fn(pred.cpu().squeeze(), target_prices, strike_prices, is_call, X.cpu().squeeze())
            
            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(input_model.parameters(), 1e-1)
            optimizer.step()
            avg_loss += loss
        loss_history.append(float(avg_loss/batch))
    return loss_history

